# OCR Evaluation Demo

This notebook demonstrates the OCR capabilities of **AWS Textract** and **OpenAI GPT-4o** on the **CORD** dataset.

We will:
1. Load samples from the CORD dataset.
2. Display the receipt images.
3. Run OCR using the selected engine.
4. Compare the prediction with the ground truth.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Add project root to sys.path to ensure we can import ocr_eval
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added {project_root} to sys.path")

import matplotlib.pyplot as plt
from PIL import Image
from dotenv import load_dotenv

from ocr_eval.data.loader import load_cord_dataset
from ocr_eval.engines.openai import OpenAIVLMEngine
from ocr_eval.engines.textract import TextractEngine
from ocr_eval.utils.metrics import calculate_wer, calculate_cer

load_dotenv()

# Initialize Engines
try:
    openai_engine = OpenAIVLMEngine()
    print("OpenAI Engine initialized.")
except Exception as e:
    print(f"Failed to init OpenAI: {e}")

try:
    textract_engine = TextractEngine()
    print("Textract Engine initialized.")
except Exception as e:
    print(f"Failed to init Textract: {e}")

In [ ]:
# Load Data
samples = load_cord_dataset(split="test", num_samples=5)
print(f"Loaded {len(samples)} samples.")

In [ ]:
def visualize_sample(sample, engine_name="openai"):
    image_path = sample["image_path"]
    ground_truth = sample["ground_truth"]
    
    # Display Image
    img = Image.open(image_path)
    plt.figure(figsize=(10, 12))
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Sample {sample['id']}")
    plt.show()
    
    # Run OCR
    print(f"Running {engine_name} OCR...")
    if engine_name == "openai":
        engine = openai_engine
    elif engine_name == "textract":
        engine = textract_engine
    else:
        print("Unknown engine")
        return
        
    try:
        prediction = engine.process_image(image_path)
        
        wer = calculate_wer(ground_truth, prediction)
        cer = calculate_cer(ground_truth, prediction)
        
        print("-" * 40)
        print(f"METRICS: WER={wer:.4f}, CER={cer:.4f}")
        print("-" * 40)
        print("PREDICTION:")
        print(prediction)
        print("-" * 40)
        print("GROUND TRUTH:")
        print(ground_truth)
        print("-" * 40)
        
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
# Run on the first sample
visualize_sample(samples[0], engine_name="openai")

In [ ]:
# Run on the second sample
visualize_sample(samples[1], engine_name="openai")